In [ ]:
import modulos as md

#lista=md.filtradodemonedas()

lista = ['BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'ALGOUSDT', 
         'KNCUSDT', 'COMPUSDT', 'DOGEUSDT', 'KAVAUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'CRVUSDT', 'SOLUSDT', 'STORJUSDT', 'UNIUSDT', 
         'AVAXUSDT', 'FTMUSDT', 'TOMOUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'AXSUSDT', 'SANDUSDT', 'XEMUSDT', 'MANAUSDT', 'LINAUSDT', 
         'MTLUSDT', '1000SHIBUSDT', 'MASKUSDT', 'DYDXUSDT', '1000XECUSDT', 'GALAUSDT', 'FLOWUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'LDOUSDT', 
         'APTUSDT', 'FETUSDT', 'RNDRUSDT', 'AGIXUSDT', 'PHBUSDT', 'CFXUSDT', 'STXUSDT', 'LQTYUSDT', 'ARBUSDT', 'SUIUSDT', '1000PEPEUSDT', 'XVGUSDT']


In [3]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
lista=['XRPUSDT',	
'XTZUSDT',	
'OMGUSDT',	
'CRVUSDT',	
'1INCHUSDT',
'MANAUSDT',
'FLOWUSDT',
'MAGICUSDT',
'MINAUSDT',
'PHBUSDT',	
'PERPUSDT',
'IDUSDT',	
'JOEUSDT',	
'HFTUSDT',	
'EDUUSDT',	
'SUIUSDT',	
'COMBOUSDT',
'MAVUSDT'	]

#lista=md.lista_de_monedas ()

#lista=['STXUSDT']

import numpy as np

for symbol in lista:    
    try:
        data = md.sigo_variacion_bitcoin(symbol)
        #data = md.estrategia_bb(symbol)
        resultado = md.backtesting(data, plot_flag = False)
        if (
            (resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor']))
            and (resultado['Return [%]']/resultado['# Trades']) >=0.33 
            ):
            print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        pass

XRPUSDT - Return [%]: 15.73% - # Trades: 8 - Profit Factor: 2.31 - Win Rate [%]: 50.0
XTZUSDT - Return [%]: 4.39% - # Trades: 8 - Profit Factor: 2.01 - Win Rate [%]: 50.0
OMGUSDT - Return [%]: 10.07% - # Trades: 6 - Profit Factor: 7.41 - Win Rate [%]: 66.66666666666666


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:462: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)



Error: local variable 'data' referenced before assignment - line: 522 - file: modulos.py - par: DEFIUSDT



c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:462: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


CRVUSDT - Return [%]: 6.81% - # Trades: 7 - Profit Factor: 3.03 - Win Rate [%]: 57.14285714285714
1INCHUSDT - Return [%]: 3.28% - # Trades: 9 - Profit Factor: 2.29 - Win Rate [%]: 55.55555555555556
MANAUSDT - Return [%]: 4.59% - # Trades: 8 - Profit Factor: 3.07 - Win Rate [%]: 62.5

Error: local variable 'data' referenced before assignment - line: 522 - file: modulos.py - par: 1000SHIBUSDT


Error: local variable 'data' referenced before assignment - line: 522 - file: modulos.py - par: BTCDOMUSDT


Error: local variable 'data' referenced before assignment - line: 522 - file: modulos.py - par: 1000XECUSDT

FLOWUSDT - Return [%]: 4.84% - # Trades: 9 - Profit Factor: 2.12 - Win Rate [%]: 55.55555555555556

Error: local variable 'data' referenced before assignment - line: 522 - file: modulos.py - par: FOOTBALLUSDT


Error: local variable 'data' referenced before assignment - line: 522 - file: modulos.py - par: 1000LUNCUSDT


Error: local variable 'data' referenced before assignment - line